In [ ]:
import numpy as np
import pandas as pd
import joblib


### MLflow – Experiment Tracking & Model Versioning


In [ ]:
!pip install mlflow


In [ ]:
import os
print("Current working directory:", os.getcwd())
print("Folders:", os.listdir())


Current working directory: C:\Users\user
Folders: ['.anaconda', '.cache', '.codeium', '.conda', '.condarc', '.continuum', '.EasyOCR', '.ipynb_checkpoints', '.ipython', '.jupyter', '.keras', '.matplotlib', '.spyder-py3', '.virtual_documents', '.vscode', '3D Objects', 'anaconda3', 'AppData', 'Application Data', 'artifacts', 'bird (1).jpg', 'Capstone Project EDA.1.ipynb', 'Car Insurance.csv', 'Case study - Unsupervised Learning (PCA).ipynb', 'cat.jpg', 'Class work Numpy & Pandas - Completed (1).ipynb', 'Comp Vision.ipynb', 'Contacts', 'Cookies', 'Dataset_ecommerce.csv', 'Desktop', 'Documents', 'Downloads', 'edb_mtk.exe', 'edb_npgsql.exe', 'edb_pem_agent.exe-20250628025359', 'edb_pem_server.exe-20250628025519', 'edb_pgagent_pg17.exe', 'edb_pgbouncer.exe', 'edb_pgjdbc.exe', 'edb_psqlodbc.exe', 'edb_psqlodbc.exe-20250628025735', 'edb_sqlprofiler_pg17.exe', 'edb_xdb_62.exe', 'edb_xdb_7.exe', 'Favorites', 'Hello World.ipynb', 'IntelGraphicsProfiles', 'Introduction to python .ipynb', 'Links', '

In [ ]:
import os

for root, dirs, files in os.walk(".", topdown=True):
    for file in files:
        if file.endswith(".pkl"):
            print(os.path.join(root, file))


.\anaconda3\envs\easyocr_env\Lib\site-packages\numpy\_core\tests\data\astype_copy.pkl
.\anaconda3\envs\Pycaret\Lib\site-packages\examples\global_fc\X.pkl
.\anaconda3\envs\Pycaret\Lib\site-packages\examples\global_fc\y.pkl
.\anaconda3\envs\Pycaret\Lib\site-packages\joblib\test\data\joblib_0.10.0_pickle_py27_np17.pkl
.\anaconda3\envs\Pycaret\Lib\site-packages\joblib\test\data\joblib_0.10.0_pickle_py33_np18.pkl
.\anaconda3\envs\Pycaret\Lib\site-packages\joblib\test\data\joblib_0.10.0_pickle_py34_np19.pkl
.\anaconda3\envs\Pycaret\Lib\site-packages\joblib\test\data\joblib_0.10.0_pickle_py35_np19.pkl
.\anaconda3\envs\Pycaret\Lib\site-packages\joblib\test\data\joblib_0.11.0_pickle_py36_np111.pkl
.\anaconda3\envs\Pycaret\Lib\site-packages\joblib\test\data\joblib_0.9.2_pickle_py27_np16.pkl
.\anaconda3\envs\Pycaret\Lib\site-packages\joblib\test\data\joblib_0.9.2_pickle_py27_np17.pkl
.\anaconda3\envs\Pycaret\Lib\site-packages\joblib\test\data\joblib_0.9.2_pickle_py33_np18.pkl
.\anaconda3\envs\Pyc

In [ ]:
import os

os.chdir(r"C:\Users\user\Desktop\Data Science Full Stack\Amdari\Intenship\Working directory")

print("Now in:", os.getcwd())
print("Artifacts contents:", os.listdir("artifacts"))


Now in: C:\Users\user\Desktop\Data Science Full Stack\Amdari\Intenship\Working directory
Artifacts contents: ['feature_cols.pkl', 'scaler.pkl', 'xgb_model.pkl']


In [ ]:
import joblib

best_xgb = joblib.load("artifacts/xgb_model.pkl")
scaler = joblib.load("artifacts/scaler.pkl")
feature_cols = joblib.load("artifacts/feature_cols.pkl")

print("✅ Loaded model, scaler and feature list successfully!")


✅ Loaded model, scaler and feature list successfully!


## Create FastAPI Inference Script

In [ ]:
from fastapi import FastAPI
import joblib
import numpy as np
import pandas as pd

app = FastAPI(title="Stock Trend Prediction API",
              description="Predict Uptrend / Sideways / Downtrend using XGBoost model",
              version="1.0")

# ================= LOAD ARTIFACTS =====================
model = joblib.load("artifacts/xgb_model.pkl")
scaler = joblib.load("artifacts/scaler.pkl")
feature_cols = joblib.load("artifacts/feature_cols.pkl")

# ================= HEALTH CHECK =======================
@app.get("/")
def home():
    return {"status": "API is running 🚀", "features_required": feature_cols}

# ================= PREDICTION ENDPOINT =================
@app.post("/predict")
def predict(data: dict):

    # Convert input into DataFrame with correct order
    df = pd.DataFrame([data])[feature_cols]

    # scale input
    X = scaler.transform(df)

    # Predict class
    pred = model.predict(X)[0]

    label_map = {0: "Downtrend 📉", 1: "Sideways ➖", 2: "Uptrend 📈"}

    return {
        "prediction_class": int(pred),
        "trend_label": label_map[pred],
        "status": "success"
    }


In [ ]:
## Setting Final Matrix

# Final evaluation numbers from your Week 2 notebook
acc_best     = 0.3889   # classification accuracy
total_return = 0.7423   # 74.23% strategy return
win_rate     = 0.4064   # 40.64% winning trades
max_drawdown = 0.9557   # 95.57% drawdown


In [ ]:
import mlflow
import mlflow.xgboost

# End any active run just in case
if mlflow.active_run():
    mlflow.end_run()

# Create/select experiment
mlflow.set_experiment("stock_trend_classification")

with mlflow.start_run(run_name="xgb_final_prod"):
    # 1) Hyperparameters
    mlflow.log_params(best_xgb.get_params())

    # 2) Metrics
    mlflow.log_metric("accuracy", acc_best)
    mlflow.log_metric("total_return", total_return)
    mlflow.log_metric("win_rate", win_rate)
    mlflow.log_metric("max_drawdown", max_drawdown)

    # 3) Model artifact
    mlflow.xgboost.log_model(best_xgb, artifact_path="xgb_model")

print("✅ MLflow run completed and model logged.")


C:\Users\user\anaconda3\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/12/06 16:45:33 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n

✅ MLflow run completed and model logged.


In [ ]:
feature_cols


['close',
 'daily_return',
 'sma_20',
 'sma_50',
 'ema_12',
 'ema_26',
 'rsi_14',
 'macd',
 'bb_width',
 'atr_14',
 'volume_ratio',
 'momentum_10']

In [ ]:
## Creating my FastAPI 

In [ ]:
from fastapi import FastAPI
import joblib
import numpy as np
import pandas as pd

# ---------- Load artifacts ----------
model = joblib.load("artifacts/xgb_model.pkl")
scaler = joblib.load("artifacts/scaler.pkl")
feature_cols = joblib.load("artifacts/feature_cols.pkl")

app = FastAPI(
    title="Stock Trend Prediction API",
    description="Predict 0=Downtrend, 1=Sideways, 2=Uptrend from technical indicators",
    version="1.0.0",
)

# ---------- Health check ----------
@app.get("/")
def health():
    return {
        "status": "ok",
        "message": "API is running",
        "required_features": feature_cols,
    }

# ---------- Prediction endpoint ----------
@app.post("/predict")
def predict(features: dict):
    """
    Expects a JSON body with keys exactly matching feature_cols.
    Example:
    {
      "close": 210.5,
      "sma_20": 205.2,
      ...
    }
    """

    # Convert dict -> DataFrame in correct column order
    df = pd.DataFrame([features])
    df = df[feature_cols]              # ensure correct ordering

    X_scaled = scaler.transform(df.values)
    probs = model.predict_proba(X_scaled)[0]
    pred_class = int(np.argmax(probs))

    label_map = {0: "Downtrend", 1: "Sideways", 2: "Uptrend"}

    return {
        "prediction_class": pred_class,
        "trend_label": label_map[pred_class],
        "probabilities": {
            "class_0": float(probs[0]),
            "class_1": float(probs[1]),
            "class_2": float(probs[2]),
        },
    }


In [ ]:
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"status": "ok", "message": "Minimal API is running"}


In [ ]:
from fastapi import FastAPI
import joblib
import numpy as np
import pandas as pd

# ================== LOAD MODEL + ARTIFACTS ==================
model = joblib.load("artifacts/xgb_model.pkl")
scaler = joblib.load("artifacts/scaler.pkl")
feature_cols = joblib.load("artifacts/feature_cols.pkl")

# ================== CREATE API ==================
app = FastAPI(
    title="Stock Trend Prediction API",
    description="Predict market trend → 0=Downtrend | 1=Sideways | 2=Uptrend",
    version="1.0"
)

# ================== ROUTE: HEALTH CHECK ==================
@app.get("/")
def home():
    return {
        "status": "running",
        "message": "Model API active",
        "features_required": feature_cols,
        "total_features": len(feature_cols)
    }

# ================== ROUTE: PREDICTION ==================
@app.post("/predict")
def predict(features: dict):

    df = pd.DataFrame([features])  # Convert input dict → DataFrame

    try:
        df = df[feature_cols]  # Reorder columns correctly
    except Exception as e:
        return {
            "error": "Feature mismatch",
            "details": str(e),
            "expected_features": feature_cols
        }

    X_scaled = scaler.transform(df.values)
    probs = model.predict_proba(X_scaled)[0]
    pred_class = int(np.argmax(probs))

    trend_label = {0: "Downtrend", 1: "Sideways", 2: "Uptrend"}[pred_class]

    return {
        "prediction_class": pred_class,
        "trend_label": trend_label,
        "probabilities": {
            "Downtrend (0)": float(probs[0]),
            "Sideways (1)": float(probs[1]),
            "Uptrend (2)": float(probs[2])
        }
    }


In [ ]:
import os
from textwrap import dedent

# 1) Make sure we're in your working directory
os.chdir(r"C:\Users\user\Desktop\Data Science Full Stack\Amdari\Intenship\Working directory")

# 2) Minimal FastAPI app code
api_code = dedent("""
    from fastapi import FastAPI

    app = FastAPI()

    @app.get("/")
    def root():
        return {"status": "ok", "message": "Minimal API is running"}
""")

# 3) Write api.py file
with open("api.py", "w", encoding="utf-8") as f:
    f.write(api_code)

print("api.py created at:", os.path.abspath("api.py"))


api.py created at: C:\Users\user\Desktop\Data Science Full Stack\Amdari\Intenship\Working directory\api.py


In [ ]:
import os
from textwrap import dedent

# Make sure you're in the working directory
os.chdir(r"C:\Users\user\Desktop\Data Science Full Stack\Amdari\Intenship\Working directory")

api_code = dedent("""
    import joblib
    import pandas as pd
    import numpy as np
    from fastapi import FastAPI

    # ===== Load Artifacts =====
    model = joblib.load("artifacts/xgb_model.pkl")
    scaler = joblib.load("artifacts/scaler.pkl")
    feature_cols = joblib.load("artifacts/feature_cols.pkl")

    app = FastAPI(
        title="Stock Trend Prediction API",
        description="Predict 0=Downtrend, 1=Sideways, 2=Uptrend using the tuned XGBoost model",
        version="1.0.0",
    )

    @app.get("/")
    def home():
        return {
            "status": "running",
            "message": "Model API active",
            "n_features": len(feature_cols),
            "features_required": feature_cols,
        }

    @app.post("/predict")
    def predict(features: dict):
        # Convert input dict to DataFrame
        df = pd.DataFrame([features])

        # Reorder columns to match training
        try:
            df = df[feature_cols]
        except KeyError as e:
            return {
                "status": "error",
                "message": f"Feature mismatch: {e}",
                "expected_features": feature_cols,
            }

        # Scale inputs
        X_scaled = scaler.transform(df.values)

        # Predict probabilities
        probs = model.predict_proba(X_scaled)[0]
        pred_class = int(np.argmax(probs))

        label_map = {0: "Downtrend", 1: "Sideways", 2: "Uptrend"}

        return {
            "status": "success",
            "prediction_class": pred_class,
            "trend_label": label_map[pred_class],
            "probabilities": {
                "class_0": float(probs[0]),
                "class_1": float(probs[1]),
                "class_2": float(probs[2]),
            },
        }
""")

with open("api.py", "w", encoding="utf-8") as f:
    f.write(api_code)

print("✅ api.py updated with model-serving API")


✅ api.py updated with model-serving API


In [ ]:
import joblib, os
os.chdir(r"C:\Users\user\Desktop\Data Science Full Stack\Amdari\Intenship\Working directory")
feature_cols = joblib.load("artifacts/feature_cols.pkl")
feature_cols


['close',
 'daily_return',
 'sma_20',
 'sma_50',
 'ema_12',
 'ema_26',
 'rsi_14',
 'macd',
 'bb_width',
 'atr_14',
 'volume_ratio',
 'momentum_10']

# Updating the log predictions for monitoring

In [ ]:
import os, textwrap

os.chdir(r"C:\Users\user\Desktop\Data Science Full Stack\Amdari\Intenship\Working directory")

# Read the current api.py
with open("api.py", "r", encoding="utf-8") as f:
    api_text = f.read()

new_predict = textwrap.dedent("""
    @app.post("/predict")
    def predict(features: dict):
        import csv
        from datetime import datetime
        import os

        # Convert input dict to DataFrame
        df = pd.DataFrame([features])

        # Reorder columns to match training
        try:
            df = df[feature_cols]
        except KeyError as e:
            return {
                "status": "error",
                "message": f"Feature mismatch: {e}",
                "expected_features": feature_cols,
            }

        # Scale inputs
        X_scaled = scaler.transform(df.values)

        # Predict probabilities
        probs = model.predict_proba(X_scaled)[0]
        pred_class = int(np.argmax(probs))

        label_map = {0: "Downtrend", 1: "Sideways", 2: "Uptrend"}
        trend_label = label_map[pred_class]

        # ===== Simple logging to CSV for monitoring =====
        os.makedirs("logs", exist_ok=True)
        log_file = os.path.join("logs", "predictions_log.csv")
        write_header = not os.path.exists(log_file)

        row = {
            "timestamp": datetime.utcnow().isoformat(),
            "prediction_class": pred_class,
            "trend_label": trend_label,
            "prob_class_0": float(probs[0]),
            "prob_class_1": float(probs[1]),
            "prob_class_2": float(probs[2]),
        }
        # add input features to the row
        for k, v in features.items():
            row[f"feat_{k}"] = v

        with open(log_file, "a", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=row.keys())
            if write_header:
                writer.writeheader()
            writer.writerow(row)
        # ================================================

        return {
            "status": "success",
            "prediction_class": pred_class,
            "trend_label": trend_label,
            "probabilities": {
                "class_0": float(probs[0]),
                "class_1": float(probs[1]),
                "class_2": float(probs[2]),
            },
        }
""")

# Replace the old predict function in api.py
import re
api_text = re.sub(r"@app.post\(\"/predict\"[\s\S]+?return \{[\s\S]+?\}\s*\n", new_predict + "\n", api_text)

with open("api.py", "w", encoding="utf-8") as f:
    f.write(api_text)

print("✅ predict() updated with logging")


✅ predict() updated with logging


## Getting the Docker for the project


In [ ]:

import os
from textwrap import dedent

os.chdir(r"C:\Users\user\Desktop\Data Science Full Stack\Amdari\Intenship\Working directory")

dockerfile = dedent("""
    FROM python:3.11-slim

    WORKDIR /app

    COPY requirements.txt .

    RUN pip install --no-cache-dir -r requirements.txt

    COPY . .

    EXPOSE 8000

    CMD ["uvicorn", "api:app", "--host", "0.0.0.0", "--port", "8000"]
""")

with open("Dockerfile", "w", encoding="utf-8") as f:
    f.write(dockerfile)

print("✅ Dockerfile created")

In [ ]:
reqs = """
fastapi
uvicorn[standard]
pandas
numpy
xgboost
scikit-learn
joblib
mlflow
"""
with open("requirements.txt", "w", encoding="utf-8") as f:
    f.write(reqs.strip() + "\n")

print("✅ requirements.txt created")

## Interactive Dashboard with Streamlit

In [ ]:
import os
from textwrap import dedent

os.chdir(r"C:\Users\user\Desktop\Data Science Full Stack\Amdari\Intenship\Working directory")

streamlit_code = dedent("""
    import streamlit as st
    import requests

    st.set_page_config(page_title="Stock Trend Dashboard", layout="centered")

    st.title("📈 Stock Trend Prediction Dashboard")
    st.write("This dashboard sends features to the FastAPI model and shows the predicted trend.")

    # Sidebar for API config
    st.sidebar.header("API Configuration")
    api_url = st.sidebar.text_input("FastAPI URL", "http://127.0.0.1:8000/predict")

    st.subheader("Input Features")

    close = st.number_input("Close price", value=210.5)
    daily_return = st.number_input("Daily return", value=0.003, format="%.5f")
    sma_20 = st.number_input("SMA 20", value=208.1)
    sma_50 = st.number_input("SMA 50", value=200.4)
    ema_12 = st.number_input("EMA 12", value=207.9)
    ema_26 = st.number_input("EMA 26", value=203.2)
    rsi_14 = st.number_input("RSI 14", value=55.3)
    macd = st.number_input("MACD", value=0.42)
    bb_width = st.number_input("Bollinger Band Width", value=0.05, format="%.4f")
    atr_14 = st.number_input("ATR 14", value=2.1)
    volume_ratio = st.number_input("Volume ratio", value=1.2)
    momentum_10 = st.number_input("Momentum 10", value=1.5)

    if st.button("Predict Trend"):
        payload = {
            "close": close,
            "daily_return": daily_return,
            "sma_20": sma_20,
            "sma_50": sma_50,
            "ema_12": ema_12,
            "ema_26": ema_26,
            "rsi_14": rsi_14,
            "macd": macd,
            "bb_width": bb_width,
            "atr_14": atr_14,
            "volume_ratio": volume_ratio,
            "momentum_10": momentum_10,
        }

        try:
            res = requests.post(api_url, json=payload, timeout=10)
            if res.status_code == 200:
                data = res.json()
                st.success(f"Predicted trend: **{data['trend_label']}** (class {data['prediction_class']})")
                st.write("Probabilities:")
                st.json(data["probabilities"])
            else:
                st.error(f"API returned status code {res.status_code}")
                st.text(res.text)
        except Exception as e:
            st.error(f"Error calling API: {e}")
""")

with open("dashboard_app.py", "w", encoding="utf-8") as f:
    f.write(streamlit_code)

print("✅ dashboard_app.py created")


In [ ]:
import os

# 1. Go to your working directory
os.chdir(r"C:\Users\user\Desktop\Data Science Full Stack\Amdari\Intenship\Working directory")

# 2. Show what's in there
print("Current working directory:", os.getcwd())
print(os.listdir())
